---
title: One-pass (Nyström) methods
description: Nyström methods for low-rank approximation of symmetric and positive semidefinite matrices
keywords: [Nyström method, generalized Nyström, symmetric matrices, positive semidefinite, kernel matrices, column sampling, eigenvalue decomposition]
numbering:
  equation:
    enumerator: 5.%s
    continue: true
  proof:theorem:
    enumerator: 5.%s
    continue: true
  proof:algorithm:
    enumerator: 5.%s
    continue: true
  proof:definition:
    enumerator: 5.%s
    continue: true
  proof:proposition:
    enumerator: 5.%s
    continue: true
---

The [Randomized SVD](alg-randomized-svd) (RSVD) and improvements require multiple passes over $\vec{A}$.
In some cases, it may be advantageous to use a method that only requires a single pass over $\vec{A}$. 


## Nyström for positive definite matrices

When $\vec{A}$ is symmetric positive semi-definite, we can use the Nyström approximation generated by a sketching matrix $\vec{\Omega}\in\R^{n\times k}$.
```{math}
:label: def-nystrom
\vec{A} \langle \vec{\Omega}\rangle := \vec{A}\vec{\Omega} ( \vec{\Omega}^\T\vec{A}\vec{\Omega})^+ \vec{\Omega}^\T\vec{A}.
```
When $\vec{\Omega}$ is a Gaussian sketching matrix, this method satisfies similar theoretical guarantees to the Randomized SVD (with respect to $k$), despite requiring half the number of matrix-vector products and only one pass over the data; see e.g. Corollary 8.2 in {cite:p}`tropp_webber_23`.


### Block Krylov methods

In fact, at the cost of more passes over the data, we can replace $\vec{\Omega}$ with a basis for a Krylov subspace. 
As noted in Lemma 5.2 in {cite:p}`tropp_webber_23` (below), the Nyström method always produces a low-rank approximation that is at least as good one-sided projection based methods [Randomized Block Krylov Iteration](./subspace-iteration-block-krylov.ipynb).


:::{prf:lemma} Nyström Helps
:label: thm-nystrom-helps
Consider a psd matrix $\vec{A} \in \R^{n \times n}$ and any matrix $\vec{M} \in \R^{n \times b}$. Then
\begin{equation*}
\|\vec{A} - \vec{A}\langle\vec{M}\rangle\|_p \leq \|\vec{A} - \vec{\Pi}_{\vec{M}} \vec{A}\|_p = \|\vec{A} - \vec{A}\vec{\Pi}_{\vec{M}}\|_p
\end{equation*}
for any Schatten $p$-norm with $1 \leq p \leq \infty$. The same inequality holds for any unitarily invariant norm.
:::


## Generalized Nyström for arbitrary matrices

A similar approach can be used for arbitrary $\vec{A}$.
Given sketching matrices $\vec{\Omega}\in\R^{d\times k_1}$ and $\vec{\Psi}\in\R^{n\times k_2}$, the Generalized Nyström approximation is the approximation
```{math}
:label: def-generalized-nystrom
\vec{A}\langle \vec{\Omega},\vec{\Psi}\rangle
:= \vec{A}\vec{\Omega} (\vec{\Psi}^\T\vec{A}\vec{\Omega})^+ \vec{\Psi}^\T\vec{A}.
```

We can understand the Generalized Nyström method as approximating the adaptive step in the Randomized SVD.
Indeed, note that the matrix $\vec{X} = \vec{A}^\T\vec{Q}$ computed by the Randomized SVD is the matrix of coefficients for the linear combination of the columns of $\vec{Q}$ that best approximates the columns of $\vec{A}$. 
That is,
```{math}
\vec{A}\vec{Q}
= 
\argmin_{\vec{X}\in\R^{b\times d}} \| \vec{A} - \vec{Q}\vec{X}^\T \|_\F.
```
Consider instead the [sketched regression](../04-Regression/sketch-and-solve.ipynb) problem
```{math}
:label: eq-nystrom-regression
\argmin_{\vec{X}\in\R^{b\times d}} \| \vec{\Phi}^\T \vec{A} - \vec{\Phi}^\T \vec{Q}\vec{X}^\T \|_\F.
```
This results in an approximation
```{math}
\vec{Q}\vec{X}^\T = \vec{Q}(\vec{\Psi}^\T\vec{Q})^+ \vec{\Phi}^\T \vec{A}
= \vec{A}\langle \vec{\Omega},\vec{\Psi}\rangle.
```

The above procedure is summarized by the following algorithm.
:::{prf:algorithm} Generalized Nyström
:label: alg-generalized-nystrom

**Input:** Matrix $A \in \R^{m \times n}$, target rank $k$, block-sizes $b,c\geq k$

1. Sample a random sketching matrix $\vec{\Omega}\sim\Call{Sketch}(n,b)$
1. Sample a random sketching matrix $\vec{\Psi}\sim\Call{Sketch}(d,c)$
1. Compute $\vec{Y} = \vec{A} \vec{\Omega}$ and $\vec{Z} = \vec{A}^\T \vec{\Psi}$
1. Compute QR factorization $\vec{Q}\vec{R} = \Call{qr}(\vec{Y})$
1. Compute $\vec{X} = (\vec{\Psi}^\T\vec{Q})^+\vec{Z}^\T$

**Output:** $\vec{Q}\vec{X}^\T$
:::


### Sketching dimension

To obtain a $(1+\varepsilon)$ approximation in the Frobenius norm (similar to {prf:ref}`thm-rsvd-frob` for the RSVD), its not too hard to show that we must solve the regression problem {prf:ref}`eq-nystrom-regression` to relative accuracy $(1+\varepsilon)$. 
Based on the analysis on [sketch-and-solve](../04-Regression/sketch-and-solve.ipynb), this requires that the sketching matrix $\vec{\Phi}$ have roughly $1/\varepsilon$ times the number of columns as $\vec{\Phi}^\T \vec{Q}$.

This is in contrast to the approximation {prf:ref}`def-nystrom` for positive semi-definite case, which works with the same sketching dimension as the Randomized SVD.